### ***Preparation: conda install bowtie2***

In [1]:
!conda search bowtie2

Jupyter detected...
2 channel Terms of Service accepted
Loading channels: - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(54, 'Connection reset by peer')': /pkgs/main/osx-arm64/repodata.json.zst

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(54, 'Connection reset by peer')': /pkgs/r/osx-arm64/repodata.json.zst

\ Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(54, 'Connection reset by peer')': /pkgs/main/noarch/repodata.json.zst

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(54, 'Connection reset by peer')': /pkgs/r/noarch/repodata.json.zst

done
# Name                       Version           Build  Channel             
bowtie2                        2.5.4      hdcd

In [2]:
# 如果没找到 bowtie2 则添加 bioconda channel
# !conda config --add channels bioconda 
# !conda config --add channels conda-forge

In [3]:
!conda list bowtie2

# packages in environment at /opt/miniconda3:
#
# Name                     Version          Build            Channel


In [4]:
# 直接在base环境中安装 bowtie2
!conda install -y bowtie2

Jupyter detected...
2 channel Terms of Service accepted
Retrieving notices: \ Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='conda.anaconda.org', port=443): Read timed out. (read timeout=5)")': /bioconda/notices.json

| Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='conda.anaconda.org', port=443): Read timed out. (read timeout=5)")': /conda-forge/notices.json

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(54, 'Connection reset by peer')': /pkgs/r/notices.json

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Read timed out. (read timeout=5)")': /pkgs/main/notices.json

R

In [6]:
!which bowtie2

/opt/miniconda3/bin/bowtie2


## ***Align ebola sequencing data against the 1976 Mayinga reference genome***

In [10]:
# 安装 NCBI entrez direct软件
# !sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"

In [15]:
# activate EDirect for this terminal session
!export PATH=${HOME}/edirect:${PATH}

In [13]:
# 创建⽬录⽤于存放参考序列
!mkdir -p Bioinfo/refs && cd Bioinfo/refs && pwd

/Users/ekeulseuji/Downloads/Bioinfo/refs


In [17]:
#⽤ efetch下载 AF086833（参考基因组）并存成 FASTA格式⽂件
# !efetch -db nuccore -format fasta -id AF086833 > AF086833.fa

In [20]:
cd Bioinfo/refs

/Users/ekeulseuji/Downloads/Bioinfo/refs


In [21]:
pwd

'/Users/ekeulseuji/Downloads/Bioinfo/refs'

In [22]:
ls

AF086833.fa


In [23]:
!which fastq-dump

fastq-dump not found


In [30]:
# 直接在base环境中安装 fastq-dump
!conda install -y sra-tools

Jupyter detected...
2 channel Terms of Service accepted
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/miniconda3

  added / updated specs:
    - sra-tools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    curl-8.15.0                |       hc25b543_0         181 KB
    fmt-11.2.0                 |       h440487c_0         173 KB  conda-forge
    icu-75.1                   |       hfee45f7_0        11.3 MB  conda-forge
    libarchive-3.8.1           | gpl_h46575ef_101         770 KB  conda-forge
    libcurl-8.15.0             |       hcdd2897_0

In [31]:
# 连服务器用 fastq-dump的话
# !ssh med-gaoxy@172.18.6.12 -p 18188
# !scp –P 21422 ⽤户名@222.28.103.184:⽬标路径  #（上传至服务器）
# !export PATH="/data01/share/Software/sratoolkit.3.0.7-centos_linux64/bin:$PATH"
'''
⽤vi打开并编辑～/.bashrc⽂件，在空⽩处添加下⾯这⾏命令
export PATH="/data01/share/Software/sratoolkit.3.0.7-centos_linux64/bin:$PATH"
⽤vi打开⽂件后，按下i或a键，切换编辑模式。完成编辑后按esc键，切换正常模式。然后按下：键，进⼊ 命令模式，输⼊wq!，然后回⻋，保存⽂件。
让设置⽣效：
'''
# !source ~/.bashrc
print("")

In [32]:
# ⽤ fastq-dump或者 fasterq-dump下载 SRR1972739的 fastq⽂件
# 出于演示⽬的仅下载前10000条 Reads 并保存为 paired-end的两个⽂件
!fastq-dump -X 10000 --split-files SRR1972739

Read 10000 spots for SRR1972739
Written 10000 spots for SRR1972739


In [34]:
# 下载完成后 看⼀下⽬录是否多了⽂件 SRR1972739_1.fastq SRR1972739_2.fastq
!ls

AF086833.fa        SRR1972739_1.fastq SRR1972739_2.fastq


### ***用 short read aligner进行 aligning*** 
#### ***1. 构建参考基因组的 index⽂件*** 

In [36]:
# 输入 FASTA 格式的参考基因组文件（或序列文件）
# AF086833 是 NCBI 登录号 即 Accession Number 代表一个特定基因或病毒基因组 将其设置为生成的索引文件的基本名称（输出前缀）
!bowtie2-build AF086833.fa AF086833

Settings:
  Output files: "AF086833.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  AF086833.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 4739
Using parameters --bmax 3555 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 3555 --dcv 1024
Constructing suffix-array element genera

In [38]:
# 看⼀下⽬录是否多了六个⽂件 
# .bt2：表示这些是 Bowtie2 格式的索引文件
# .rev：表示索引包含参考基因组的反向互补链 即 Reverse Complement 因为测序 reads 可能来自任一链
# 索引文件通过特殊的基于 Burrows-Wheeler 变换 即 BWT 的数据结构 将基因组信息高度压缩和优化
# Bowtie2 可以利用这个索引进行高效的随机访问 将比对时间从几小时缩短到几分钟
!ls

AF086833.1.bt2     AF086833.4.bt2     AF086833.rev.2.bt2
AF086833.2.bt2     AF086833.fa        SRR1972739_1.fastq
AF086833.3.bt2     AF086833.rev.1.bt2 SRR1972739_2.fastq


#### ***2.开始比对*** 

##### `-x` ***参考基因组索引***
##### `-1`/`-2` ***双端测序reads***
##### `-S` ***输出.SAM文件***
##### `2>` ***重定向统计信息到文件***

In [41]:
!bowtie2 -x AF086833 \
-1 SRR1972739_1.fastq \
-2 SRR1972739_2.fastq \
> bowtie2_output.sam

10000 reads; of these:
  10000 (100.00%) were paired; of these:
    5086 (50.86%) aligned concordantly 0 times
    4914 (49.14%) aligned concordantly exactly 1 time
    0 (0.00%) aligned concordantly >1 times
    ----
    5086 pairs aligned concordantly 0 times; of these:
      827 (16.26%) aligned discordantly 1 time
    ----
    4259 pairs aligned 0 times concordantly or discordantly; of these:
      8518 mates make up the pairs; of these:
        7463 (87.61%) aligned 0 times
        1055 (12.39%) aligned exactly 1 time
        0 (0.00%) aligned >1 times
62.69% overall alignment rate


In [ ]:
# 总体⽐对率 62.69%（成功⽐对的reads⽐例）
# ⼀致⽐对 49.14%（双端reads正确配对⽐对）

In [43]:
!cat bowtie2_output.sam | grep -v "^@" | head -4 | cut -f 1-6

SRR1972739.1	137	AF086833.2	15600	40	101M
SRR1972739.1	69	AF086833.2	15600	0	*
SRR1972739.2	89	AF086833.2	4919	42	101M
SRR1972739.2	165	AF086833.2	4919	0	*
grep: stdout: Broken pipe
